In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv


In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax"

import polars as pl
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

from centimators.model_estimators import MLPRegressor
from centimators.keras_cortex import KerasCortex

In [3]:
X, y = make_regression(
    n_samples=2000,
    n_features=20,
    noise=0.1,
    random_state=42,
)

X = pl.DataFrame(X)
y = pl.Series(y)

# train / val / test split  (60 / 20 / 20)
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.5, random_state=42
)

print(X_train.shape, X_val.shape, X_test.shape)

(1200, 20) (400, 20) (400, 20)


In [4]:
base_mlp = MLPRegressor(
    hidden_units=(64, 32),
    dropout_rate=0.1,
)

cortex = KerasCortex(
    base_estimator=base_mlp,
    n_iterations=5,
    lm="openai/gpt-4o-mini",
    verbose=True
)

In [5]:
cortex.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=256,
)

Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - loss: 30056.2285 - mse: 30056.2285
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30656.7637 - mse: 30656.7637  
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31089.9023 - mse: 31089.9023
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29419.2422 - mse: 29419.2422
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29960.8262 - mse: 29960.8262
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step

--- Iteration 1 ---
Reasoning: 
To improve validation scores, we can consider several modifications to the model architecture and training process. One effective approach is to add batch normalization layers after the dense layers. This can help stabilize and accelerate training by normalizing the inputs to each layer. Additionally, we can experiment with different activation functions, such as 'relu' or 'elu', which may lead to better performance compared to the current activation. Finally, we can also consider increasing the drop

KerasCortex(base_estimator=MLPRegressor(output_units=1,
                                        optimizer=<class 'keras.src.optimizers.adam.Adam'>,
                                        learning_rate=0.001,
                                        loss_function='mse',
                                        metrics=['mse'],
                                        model=None,
                                        distribution_strategy=None,
                                        hidden_units=(64, 32),
                                        activation='relu',
                                        dropout_rate=0.1),
            lm=<dspy.clients.lm.LM object at 0x7fe2689949e0>, verbose=True)